In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from nilearn.decoding import Decoder
from nilearn.image import load_img
from sklearn.model_selection import LeaveOneGroupOut

In [ ]:
BIDS_ROOT = Path(r"C:/Users/duart/Desktop/Tese/Mapping_Tese/mapping_tese/data/BIDS-somatosensory/BIDS-somatosensory")
DERIVATIVES = BIDS_ROOT / "derivatives" / "fmriprep"


subject = "sub-p0001"
session = "ses-01"
task = "task-S1Map"
space = "MNI152NLin2009cAsym"

n_runs = 4
TR = 6.0

### Event files all runs

In [ ]:
all_events = []
for run in range(1, n_runs + 1):
    events_path = BIDS_ROOT / subject / session / "func" / f"{subject}_{session}_{task}_run-{run}_events.tsv"
    events = pd.read_csv(events_path, sep='\t')
    events['run'] = run
    all_events.append(events)

events_df = pd.concat(all_events, ignore_index=True)

# remove Baseline and Jitter
stim_events = events_df[~events_df['trial_type'].isin(['Baseline', 'Jitter'])].copy()

print(f"Total events loaded: {len(events_df)}")
print(f"Stimulation events: {len(stim_events)}")
print(f"Unique conditions: {stim_events['trial_type'].nunique()}")
print(f"\nConditions: {sorted(stim_events['trial_type'].unique())}")
print(f"\nSamples per condition:")
print(stim_events['trial_type'].value_counts().sort_index())

In [ ]:
from nilearn.image import index_img, mean_img, concat_imgs
from nilearn.image import load_img

HRF_DELAY = 5.0
WINDOW = 2

sample_imgs = []
labels = []

for run in range(1, n_runs + 1):

    func_path = (DERIVATIVES / subject / session / "func" /
                 f"{subject}_{session}_{task}_run-{run}_space-{space}_desc-preproc_bold.nii.gz")

    img = load_img(str(func_path))
    run_events = stim_events[stim_events['run'] == run]

    run_length = img.shape[3]

    print(f"Run {run} loaded: {img.shape}")

    for _, event in run_events.iterrows():

        peak_volume = int((event['onset'] + HRF_DELAY) / TR)
        if peak_volume >= run_length:
            continue

        vols = list(range(max(0, peak_volume-WINDOW),
                          min(run_length, peak_volume+WINDOW+1)))

        window_img = index_img(img, vols)
        averaged = mean_img(window_img)

        sample_imgs.append(averaged)
        labels.append(event['trial_type'])

# build final dataset (small!)
X = concat_imgs(sample_imgs)
y = np.array(labels)

print("\nFinal dataset shape:", X.shape)
print("Samples:", len(y))
print("Classes:", len(np.unique(y)))


In [ ]:
# brain mask from first run -> all the same in MNI space
mask_path = (DERIVATIVES / subject / session / "func" / 
             f"{subject}_{session}_{task}_run-1_space-{space}_desc-brain_mask.nii.gz")
mask_img = load_img(str(mask_path))

print(f"Mask shape: {mask_img.shape}")
print(f"Mask loaded: {mask_path.name}")

In [ ]:
from nilearn.image import load_img, index_img, mean_img

HRF_DELAY = 5.0   # seconds
WINDOW = 2        # ±2 volumes

sample_imgs = []
labels = []

for run in range(1, n_runs + 1):

    func_path = (DERIVATIVES / subject / session / "func" /
                 f"{subject}_{session}_{task}_run-{run}_space-{space}_desc-preproc_bold.nii.gz")

    img = load_img(str(func_path))
    run_events = stim_events[stim_events['run'] == run]
    run_length = img.shape[3]

    print(f"Run {run}: {img.shape}")

    for _, event in run_events.iterrows():

        # HRF peak timing
        peak_volume = int((event['onset'] + HRF_DELAY) / TR)

        if peak_volume >= run_length:
            continue

        # temporal averaging window
        vols = list(range(max(0, peak_volume-WINDOW),
                          min(run_length, peak_volume+WINDOW+1)))

        window_img = index_img(img, vols)
        averaged_img = mean_img(window_img)

        sample_imgs.append(averaged_img)
        labels.append(event['trial_type'])

y = np.array(labels)

print(f"\nTotal samples: {len(y)}")
print(f"Unique labels: {len(np.unique(y))}")
print(f"Labels: {np.unique(y)}")


In [ ]:
from nilearn.image import concat_imgs

X = concat_imgs(sample_imgs)

print("Final dataset shape:", X.shape)


In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

print("Cross-validation: Stratified 5-fold")


In [ ]:
from nilearn.datasets import fetch_atlas_harvard_oxford
from nilearn.image import math_img

atlas = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
s1_index = atlas.labels.index('Postcentral Gyrus')
s1_mask = math_img(f"img == {s1_index}", img=atlas.maps)
print("S1 mask created")

In [ ]:
decoder = Decoder(
    estimator='svc',           
    mask=s1_mask,             
    standardize='zscore_sample',  
    screening_percentile=20,   
    cv=cv,                     
    n_jobs=-1  ,
    scoring='accuracy',             
)

decoder.fit(X, y)

In [ ]:
cv_scores = decoder.cv_scores_

# Calculate statistics
n_conditions = len(np.unique(y))

print("=" * 70)
print("DECODING RESULTS")
print("=" * 70)
print(f"\nAccuracy per fold:")
for fold, (condition, scores) in enumerate(cv_scores.items(), 1):
    print(f"  Fold {fold}: {np.mean(scores):.3f}")

all_scores = np.concatenate([scores for scores in cv_scores.values()])
mean_accuracy = np.mean(all_scores)
std_accuracy = np.std(all_scores)

print(f"\n{'=' * 70}")
print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")

print(f"Performance:   {mean_accuracy*100:.1f}%")
print(f"{'=' * 70}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

fold_scores = [np.mean(scores) for scores in cv_scores.values()]
folds = range(1, len(fold_scores) + 1)

ax.bar(folds, fold_scores, color='steelblue', alpha=0.7, edgecolor='black', label='Fold accuracy')
ax.axhline(y=mean_accuracy, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_accuracy:.3f}')

ax.set_xlabel('Fold (Run Left Out)', fontsize=12, fontweight='bold')
ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Cross-Validation Decoding Accuracy', fontsize=14, fontweight='bold')
ax.set_xticks(folds)
ax.set_ylim([0, max(fold_scores) * 1.2])
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from nilearn import plotting

coef_img = decoder.coef_img_['E1'] 
weight_imgs = [decoder.coef_img_[condition] for condition in np.unique(y)]
fig = plt.figure(figsize=(12, 4))
plotting.plot_stat_map(
    coef_img,
    title=f'Feature Weights - Condition E1',
    cut_coords=5,
    display_mode='z',
    cmap='cold_hot',
    figure=fig
)

plt.show()

In [ ]:
from scipy.stats import binomtest

n_conditions = len(np.unique(y))
chance_level = 1.0 / n_conditions
total_samples = len(y)
correct_predictions = int(mean_accuracy * total_samples)

#H0 = random classification at chance level
p_value = binomtest(correct_predictions, total_samples, chance_level, alternative='greater')

print("=" * 70)
print("STATISTICAL SIGNIFICANCE TESTING")
print("=" * 70)
print(f"\nChance level (1/n_conditions): {chance_level:.3f} ({chance_level*100:.1f}%)")
print(f"Total samples: {total_samples}")
print(f"Observed correct predictions: {correct_predictions}")
print(f"Expected correct predictions (chance): {int(total_samples * chance_level)}")
print(f"\nBinomial Test (one-tailed, greater than chance):")
print(f"  p-value: {p_value.pvalue:.2e}")
if p_value.pvalue < 0.001:
    print(f"  Result: Significantly above chance (p < 0.001) ***")
elif p_value.pvalue < 0.05:
    print(f"  Result: Significantly above chance (p < 0.05) *")
else:
    print(f"  Result: Not significantly above chance (p = {p_value.pvalue:.3f})")


In [ ]:
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.base import clone


# images->voxel features
X_features = decoder.masker_.transform(X)
clf = SVC(kernel='linear')
y_pred = np.zeros_like(y, dtype=object)

for train, test in cv.split(X_features, y):
    model = clone(clf)
    model.fit(X_features[train], y[train])
    y_pred[test] = model.predict(X_features[test])

conditions = np.unique(y)
cm = confusion_matrix(y, y_pred, labels=conditions)
per_condition_accuracy = cm.diagonal() / cm.sum(axis=1)


print("=" * 70)
print("PER-CONDITION ACCURACY")
print("=" * 70)


for i, condition in enumerate(conditions):
    acc = per_condition_accuracy[i]
    samples = cm[i].sum()
    print(f"{condition}: {acc:.3f} ({acc*100:.1f}%) - {samples} samples")

print(f"\n{'=' * 70}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y, y_pred):.3f}")
print(f"{'=' * 70}")

fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(cm, cmap='Blues', aspect='auto')

ax.set_xticks(np.arange(len(conditions)))
ax.set_yticks(np.arange(len(conditions)))
ax.set_xticklabels(conditions, rotation=45, ha='right')
ax.set_yticklabels(conditions)

ax.set_xlabel('Predicted Condition', fontsize=12, fontweight='bold')
ax.set_ylabel('True Condition', fontsize=12, fontweight='bold')
ax.set_title('Confusion Matrix - Somatotopic Decoding', fontsize=14, fontweight='bold')

for i in range(len(conditions)):
    for j in range(len(conditions)):
        text = ax.text(j, i, cm[i, j],
                      ha="center", va="center", color="black", fontsize=9)

plt.colorbar(im, ax=ax, label='Number of Samples')
plt.tight_layout()
plt.show()


In [ ]:
from nilearn.masking import apply_mask, unmask

cov = np.cov(X_features, rowvar=False)
pattern_imgs = {}
for condition, weight_img in decoder.coef_img_.items():
    w = apply_mask(weight_img, decoder.mask_img_)
    pattern = cov @ w #-> Haufe et al. (2014) transformation to get activation patterns
    pattern_imgs[condition] = unmask(pattern, decoder.mask_img_)

# subset of conditions to visualize (first 6)
n_to_plot = min(6, len(conditions))
selected_conditions = conditions[:n_to_plot]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, condition in enumerate(selected_conditions):
    ax = axes[idx]
    coef_img = pattern_imgs[condition]
    vmax = np.percentile(np.abs(apply_mask(list(pattern_imgs.values()), decoder.mask_img_)), 99)
    
    plotting.plot_stat_map(
        coef_img,
        title=f'Pattern - {condition}',
        cut_coords=3,
        display_mode='z',
        cmap='RdBu_r',
        figure=fig,
        axes=ax,
        colorbar=False,
        threshold=None,
        vmax=vmax,
        symmetric_cbar=True,
    )

for idx in range(n_to_plot, len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Somatotopic Activation Patterns for Selected Conditions (Haufe-transformed)', 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()


In [ ]:
# Diagnostic: Check weight statistics and thresholding
print("\n" + "="*70)
print("WEIGHT STATISTICS FOR EACH CONDITION")
print("="*70)

for condition in conditions[:5]:
    coef_img = decoder.coef_img_[condition]
    coef_data = coef_img.get_fdata()
    print(f"\n{condition}:")
    print(f"  Min weight: {coef_data.min():.4f}")
    print(f"  Max weight: {coef_data.max():.4f}")
    print(f"  Mean weight: {coef_data.mean():.4f}")
    print(f"  Std weight: {coef_data.std():.4f}")
    print(f"  Voxels with |weight| > 0.1: {np.sum(np.abs(coef_data) > 0.1)}")
    print(f"  Voxels with |weight| > 0.5: {np.sum(np.abs(coef_data) > 0.5)}")


In [ ]:
# Load anatomical reference image for better localization
anat_path = (DERIVATIVES / subject / session / "anat" / 
             f"{subject}_{session}_space-{space}_desc-preproc_T1w.nii.gz")
anat_img = load_img(str(anat_path))
print(f"Anatomical image loaded: {anat_path.name}")
print(f"Shape: {anat_img.shape}")


In [ ]:
# Improved weight visualization with anatomical reference and thresholding
# Plot with thresholding to highlight significant weights
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

threshold_percentile = 90

for idx, condition in enumerate(selected_conditions):
    ax = axes[idx]
    coef_img = decoder.coef_img_[condition]
    coef_data = coef_img.get_fdata()
    abs_weights = np.abs(coef_data)
    threshold = np.percentile(abs_weights[abs_weights > 0], threshold_percentile)
    
    plotting.plot_stat_map(
        coef_img,
        bg_img=anat_img, 
        title=f'Weights - {condition}\n(threshold: {threshold:.4f})',
        cut_coords=[0, -25, 5],  # central region, include postcentral area
        display_mode='ortho',  # multiple views
        cmap='RdBu_r',
        figure=fig,
        axes=ax,
        colorbar=True,
        threshold=threshold
    )

for idx in range(n_to_plot, len(axes)):
    axes[idx].set_visible(False)

plt.suptitle(f'Classifier Weights with Anatomical Reference\n(90th percentile threshold)', 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print(f"Thresholded visualization with anatomical background:")
print(f"Using 90th percentile of absolute weights to highlight significant activations")


In [ ]:
from scipy.ndimage import find_objects, label

print("\n" + "="*70)
print("PEAK ACTIVATION LOCATIONS (MNI COORDINATES)")
print("="*70)

for condition in conditions[:5]:
    coef_img = decoder.coef_img_[condition]
    coef_data = coef_img.get_fdata()
    affine = coef_img.affine
    
    # Find voxels in the top 5% of absolute weights
    abs_weights = np.abs(coef_data)
    threshold = np.percentile(abs_weights[abs_weights > 0], 95)
    mask = abs_weights > threshold
    
    if np.sum(mask) > 0:
        coords = np.where(mask)
        peak_idx = np.argmax(abs_weights[mask])
        peak_voxel = (coords[0][peak_idx], coords[1][peak_idx], coords[2][peak_idx])
        
        # Convert voxel to MNI coordinates
        peak_mni = affine @ np.array([peak_voxel[0], peak_voxel[1], peak_voxel[2], 1])
        
        print(f"\n{condition}:")
        print(f"  Peak voxel (array coords): {peak_voxel}")
        print(f"  Peak MNI coords: x={peak_mni[0]:.1f}, y={peak_mni[1]:.1f}, z={peak_mni[2]:.1f}")
        print(f"  Peak weight value: {coef_data[peak_voxel]:.4f}")
        print(f"  Number of suprathreshold voxels: {np.sum(mask)}")
        
        x_mni = peak_mni[0]
        if x_mni < -5:
            hemisphere = "LEFT"
        elif x_mni > 5:
            hemisphere = "RIGHT"
        else:
            hemisphere = "MIDLINE"
        print(f"  Hemisphere: {hemisphere}")
    else:
        print(f"\n{condition}: No significant weights found")


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fold_scores = [np.mean(scores) for scores in cv_scores.values()]
ax = axes[0]
ax.bar(range(1, len(fold_scores) + 1), fold_scores, 
       color='steelblue', alpha=0.7, edgecolor='black', label='Fold accuracy')
ax.axhline(y=mean_accuracy, color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {mean_accuracy:.3f}')
ax.axhline(y=chance_level, color='gray', linestyle=':', linewidth=2, 
           label=f'Chance: {chance_level:.3f}')
ax.set_xlabel('Cross-Validation Fold (Run Left Out)', fontsize=11, fontweight='bold')
ax.set_ylabel('Accuracy', fontsize=11, fontweight='bold')
ax.set_title('CV Fold Stability', fontsize=12, fontweight='bold')
ax.set_xticks(range(1, len(fold_scores) + 1))
ax.legend()
ax.grid(axis='y', alpha=0.3)

ax = axes[1]
fold_scores_array = np.array(fold_scores)
stats_data = {
    'Mean': mean_accuracy,
    'Std Dev': std_accuracy,
    'Min': np.min(fold_scores_array),
    'Max': np.max(fold_scores_array),
    'Range': np.max(fold_scores_array) - np.min(fold_scores_array)
}

y_pos = np.arange(len(stats_data))
values = list(stats_data.values())
colors_stats = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

bars = ax.barh(y_pos, values, color=colors_stats, alpha=0.7, edgecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(stats_data.keys()))
ax.set_xlabel('Value', fontsize=11, fontweight='bold')
ax.set_title('Cross-Validation Performance Statistics', fontsize=12, fontweight='bold')

for i, (bar, value) in enumerate(zip(bars, values)):
    ax.text(value, bar.get_y() + bar.get_height()/2, f'{value:.3f}', 
            ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("=" * 70)
print("CROSS-VALIDATION STABILITY")
print("=" * 70)
print(f"Mean Accuracy:        {mean_accuracy:.3f}")
print(f"Standard Deviation:   {std_accuracy:.3f}")
print(f"Min Accuracy:         {np.min(fold_scores_array):.3f}")
print(f"Max Accuracy:         {np.max(fold_scores_array):.3f}")
print(f"Range:                {np.max(fold_scores_array) - np.min(fold_scores_array):.3f}")
print(f"Coefficient of Variation: {std_accuracy / mean_accuracy:.3f}")
print(f"{'=' * 70}")


## Summary Report

In [ ]:
print("\n" + "="*70)
print("DECODING ANALYSIS SUMMARY")
print("="*70)
print(f"\nParticipant: {subject}, Session: {session}")
print(f"Task: {task}")
print(f"Number of runs: {n_runs}")
print(f"Number of electrodes (conditions): {n_conditions}")
print(f"Total samples: {len(y)}")
print(f"\nCLASSIFIER SETUP:")
print(f"  Model: Support Vector Classifier (Linear kernel)")
print(f"  Feature standardization: Z-score normalization")
print(f"  Feature selection: 20th percentile screening")
print(f"  Cross-validation: Leave-One-Group-Out (by run)")
print(f"\nRESULTS:")
print(f"  Mean Decoding Accuracy: {mean_accuracy:.3f} ({mean_accuracy*100:.1f}%)")
print(f"  Chance Level: {chance_level:.3f} ({chance_level*100:.1f}%)")
print(f"  Statistical Significance: p < 0.001 ***" if p_value.pvalue < 0.001 else f"  Statistical Significance: p = {p_value.pvalue:.3f}")
print(f"  Balanced Accuracy: {balanced_accuracy_score(y, y_pred):.3f}")
print(f"\nCONSISTENCY:")
print(f"  Std Dev (CV folds): {std_accuracy:.3f}")
print(f"  Min Fold Accuracy: {np.min(np.array(fold_scores)):.3f}")
print(f"  Max Fold Accuracy: {np.max(np.array(fold_scores)):.3f}")
print(f"\nINTERPRETATION:")
if mean_accuracy > (0.5 * chance_level):
    if p_value.pvalue < 0.001:
        print(f"  ✓ Electrode-specific activations are highly decodable from fMRI")
    elif p_value.pvalue < 0.05:
        print(f"  ✓ Electrode-specific activations are significantly decodable from fMRI")
    else:
        print(f"  • Moderate decodability, but not statistically significant")
else:
    print(f"  • Poor decodability; performance near chance level")

print("="*70 + "\n")
